# Data Science Final Project

In [1]:
# !pip install --user --upgrade nltk
# !pip install --user --upgrade keras
# !pip install --user --upgrade tensorflow
# !pip install --user --upgrade textblob
from matplotlib import pyplot
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from datetime import datetime
import time
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import gc
from dateutil import parser
import string
import keras
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from sklearn.linear_model import SGDClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.utils import pad_sequences


plt.rcParams.update(plt.rcParamsDefault)
%matplotlib inline
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tdepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
dfreviews = pd.read_csv('Train.csv')
print(dfreviews.columns)
dfreviews.head()

Index(['text', 'label'], dtype='object')


,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


https://www.kaggle.com/datasets/kazanova/sentiment140

https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/

https://www.kaggle.com/datasets/columbine/imdb-dataset-sentiment-analysis-in-csv-format

In [2]:
df = pd.read_csv('twitterdata.csv', encoding = "ISO-8859-1")
print('Loaded file.')
df = pd.DataFrame(np.vstack([df.columns, df])) # Moves column names into row 1
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'tweet'] # Renames columns
df.replace({'sentiment': {4: 1}}, inplace=True) # Replaces all '4's with '1's in column 'sentiment'
df.replace({'sentiment': {0: -1}}, inplace=True) # Replaces all '0's with '-1's in column 'sentiment'
data_types_dict = {
    'sentiment': int,
    'id': float
}
print('Fixed sentiment values.')

df = df.astype(data_types_dict)
df.dtypes

Loaded file.
Fixed sentiment values.


sentiment      int32
id           float64
date          object
query         object
user          object
tweet         object
dtype: object

In [3]:
df = df.sample(frac=1).reset_index(drop=True) # shuffles rows so its not all 1 and then -1
df = df[['sentiment', 'tweet']]
df.head()

,sentiment,tweet
0,1,Just got to our LA campus...love the energy in...
1,1,Tamed that weave into a ponytail. Oh yes 13day...
2,1,@hartluck LOL UR too cute. Thats 4 sure @pink ...
3,1,tOmOrrOw new WeeK !!! ^^ 26 Daays fOr cOncert...
4,1,@SeandBlogonaut You just saved your Twitter so...


In [4]:
s = df.stack().value_counts()
print(s)

1                                                                                                                                800000
-1                                                                                                                               799999
isPlayer Has Died! Sorry                                                                                                            210
good morning                                                                                                                        118
headache                                                                                                                            115
                                                                                                                                  ...  
didnt go in college today, heeaaddaachhee.  i had 2 weird dreams last night                                                           1
@tantonik Sonic!!!!!!!!!!!!!!!!  You're the seco

In [5]:
# for i in range(10):
#     print(df['tweet'][i])
content = df['tweet'][:5000].values.tolist()
labels = df['sentiment'][:5000].values.tolist()
content = [x.strip() for x in content] # Deletes white space before and after
content[:20]

['Just got to our LA campus...love the energy in this place',
 "Tamed that weave into a ponytail. Oh yes 13dayss i'm so happy",
 '@hartluck LOL UR too cute. Thats 4 sure @pink is the hottest girl ever  U Lucky Lucky man lol, well shes pretty lucky 2 have U as well',
 'tOmOrrOw new WeeK !!! ^^  26 Daays fOr cOncert  teQ',
 '@SeandBlogonaut You just saved your Twitter soul.',
 "David Eddings is dead. I hope he died believing he'd gotten that one story finally told.",
 '@mileycyrus i voted for you mileyyy',
 "I'm going to bed! I'm beat out after today, had a crazy water balloon fight! Best last day of school ever!  Going to miss everyone!",
 'BAYLEIGH AND I HAZ PIZZA AND BREADSTICKS.',
 'http://twitpic.com/7yb5u - came across this picture this morning, how i miss the production',
 "I know.  i'm really sad. I miss him so much. I hope he comes down again soon",
 '@ElizaPatricia time for my bed too! 7am here and just finished a 12hr shift',
 '@ibosmiley yep. got out of the market last fall. 

In [6]:
y = np.array(labels, dtype='int8')
y[:20]

array([ 1,  1,  1,  1,  1, -1,  1, -1,  1, -1, -1, -1,  1, -1, -1, -1,  1,
        1, -1, -1], dtype=int8)

In [54]:
# REWORKED WITH REVIEWS DATASET
df = dfreviews
content = df['text'].values.tolist()
labels = df['label'].values.tolist()
content = [x.strip() for x in content] # Deletes white space before and after
print(content[:20])
y = np.array(labels, dtype='int8')
print(y[:20])

['I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.', "When I put this movie in my DVD player, and sat down with a coke and some chips, I had some expectations. I was hoping t

## stop words #1

In [55]:
def full_remove(x, removal_list):
    for w in removal_list:
        x = x.replace(w, ' ')
    return x

## Remove digits
digits = [str(x) for x in range(10)]
remove_digits = [full_remove(x, digits) for x in content]

## Remove punctuation
remove_punc = [full_remove(x, list(string.punctuation)) for x in remove_digits]

## Make everything lower-case and remove any white space
sents_lower = [x.lower() for x in remove_punc]
sents_lower = [x.strip() for x in sents_lower]

## Remove stop words
from nltk.corpus import stopwords
stops = stopwords.words("English")
def removeStopWords(stopWords, txt):
    newtxt = ' '.join([word for word in txt.split() if word not in stopWords])
    return newtxt
content1 = [removeStopWords(stops,x) for x in sents_lower]
content1[:20]

['grew b watching loving thunderbirds mates school watched played thunderbirds school lunch school wanted virgil scott one wanted alan counting became art form took children see movie hoping would get glimpse loved child bitterly disappointing high point snappy theme tune could compare original score thunderbirds thankfully early saturday mornings one television channel still plays reruns series gerry anderson wife created jonatha frakes hand directors chair version completely hopeless waste film utter rubbish cgi remake may acceptable replacing marionettes homo sapiens subsp sapiens huge error judgment',
 'put movie dvd player sat coke chips expectations hoping movie would contain strong points first movie awsome animation good flowing story excellent voice cast funny comedy kick ass soundtrack disappointment found atlantis milo return read reviews first might let following paragraph directed seen first movie enjoyed primarily points mentioned br br first scene appears shock picked at

## stops we defined instead

In [8]:
stop_set = ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']
content2 = [removeStopWords(stop_set,x) for x in sents_lower]
content2[:20]

['just got our la campus love energy in this place',
 'tamed that weave into ponytail oh yes dayss m so happy',
 'hartluck lol ur too cute thats sure pink is hottest girl ever u lucky lucky man lol well shes pretty lucky have u as well',
 'tomorrow new week daays for concert teq',
 'seandblogonaut you just saved your twitter soul',
 'david eddings is dead hope died believing d gotten that one story finally told',
 'mileycyrus voted for you mileyyy',
 'm going bed m beat out after today had crazy water balloon fight best last day school ever going miss everyone',
 'bayleigh and haz pizza and breadsticks',
 'http twitpic com yb u came across this picture this morning how miss production',
 'know m really sad miss him so much hope comes down again soon',
 'elizapatricia time for my bed too am here and just finished hr shift',
 'ibosmiley yep got out market last fall whew',
 'ahhhhh andrea bocelli con te partiro reminds me my friend who died â\x99« http blip fm s aj',
 'sweetwhirl you too 

## stemming

In [9]:
import nltk
def stemporter(words):
    porter = nltk.PorterStemmer()
    new_words = [porter.stem(w) for w in words]
    return new_words
    
def stemlancaster(words):
    porter = nltk.LancasterStemmer()
    new_words = [porter.stem(w) for w in words]
    return new_words    

porter = [stemporter(x.split()) for x in content2]
porter = [" ".join(i) for i in porter]
porter[0:10]

['just got our la campu love energi in thi place',
 'tame that weav into ponytail oh ye dayss m so happi',
 'hartluck lol ur too cute that sure pink is hottest girl ever u lucki lucki man lol well she pretti lucki have u as well',
 'tomorrow new week daay for concert teq',
 'seandblogonaut you just save your twitter soul',
 'david ed is dead hope die believ d gotten that one stori final told',
 'mileycyru vote for you mileyyy',
 'm go bed m beat out after today had crazi water balloon fight best last day school ever go miss everyon',
 'bayleigh and haz pizza and breadstick',
 'http twitpic com yb u came across thi pictur thi morn how miss product']

## vectorizing

In [10]:
vectorizer = CountVectorizer(analyzer = "word", 
                             preprocessor = None, 
                             stop_words =  'english', 
                             max_features = 6000, ngram_range=(1,5))
data_features = vectorizer.fit_transform(content2)
tfidf_transformer = TfidfTransformer()
data_features_tfidf = tfidf_transformer.fit_transform(data_features)
data_mat = data_features_tfidf.toarray()

np.random.seed(0)
test_index = np.append(np.random.choice((np.where(y==-1))[0], 250, replace=False), np.random.choice((np.where(y==1))[0], 250, replace=False))
train_index = list(set(range(len(labels))) - set(test_index))
train_data = data_mat[train_index,]
train_labels = y[train_index]
test_data = data_mat[test_index,]
test_labels = y[test_index]

#Create polarity function and subjectivity function
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity
pol_list = [pol(x) for x in content2]
sub_list = [sub(x) for x in content2]

In [11]:
for i in range(10):
    print(content2[i], '\t', pol_list[i], sub_list[i])

just got our la campus love energy in this place 	 0.5 0.6
tamed that weave into ponytail oh yes dayss m so happy 	 0.8 1.0
hartluck lol ur too cute thats sure pink is hottest girl ever u lucky lucky man lol well shes pretty lucky have u as well 	 0.41666666666666674 0.7876543209876543
tomorrow new week daays for concert teq 	 0.13636363636363635 0.45454545454545453
seandblogonaut you just saved your twitter soul 	 0.0 0.0
david eddings is dead hope died believing d gotten that one story finally told 	 -0.1 0.7
mileycyrus voted for you mileyyy 	 0.0 0.0
m going bed m beat out after today had crazy water balloon fight best last day school ever going miss everyone 	 0.13333333333333333 0.4222222222222222
bayleigh and haz pizza and breadsticks 	 0.0 0.0
http twitpic com yb u came across this picture this morning how miss production 	 0.0 0.0


In [12]:
## Fit logistic classifier on training data
clf = SGDClassifier(loss="log", penalty="none")
clf.fit(train_data, train_labels)
## Pull out the parameters (w,b) of the logistic regression model
w = clf.coef_[0,:]
b = clf.intercept_
## Get predictions on training and test data
preds_train = clf.predict(train_data)
preds_test = clf.predict(test_data)
## Compute errors
errs_train = np.sum((preds_train > 0.0) != (train_labels > 0.0))
errs_test = np.sum((preds_test > 0.0) != (test_labels > 0.0))
print("Training error: ", float(errs_train)/len(train_labels))
print("Test error: ", float(errs_test)/len(test_labels))

Training error:  0.04377777777777778
Test error:  0.332


In [13]:
## Convert vocabulary into a list:
vocab = np.array([z[0] for z in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])])
## Get indices of sorting w
inds = np.argsort(w)
## Words with large negative values
neg_inds = inds[0:50]
print("Highly negative words: ")
# MB: fixed bug here
print([x for x in list(vocab[neg_inds])])
## Words with large positive values
pos_inds = inds[-49:-1]
print("Highly positive words: ")
print([x for x in list(vocab[pos_inds])])

Highly negative words: 
['sad', 'sick', 'sucks', 'ugh', 'bummer', 'ergh', 'cold', 'jealous', 'hate', 'worst', 'forks', 'miss', 'fml', 'wont', 'awake', 'poor', 'gh', 'headache', 'aw', 'er', 'just wants', 'missing', 'lastfm', 'didn', 'mad', 'terrible', 'lmfao', 'feels like', 'housework', 'pool', 'ouch', 'surgery', 'aches', 'rude', 'jonasbrothers', 'upset', 'missed', 'near', 'means', 'feel', 'wednesday', 'unfortunately', 'human', 'everytime', 'revision', 'doing homework', 'download', 'wishing', 'pushing', 'isnt']
Highly positive words: 
['love', 'new', 'supermarket', 'send', 'getting ready', 'know haha', 'hi', 'em', 'gettin', 'fireworks', 'smile', 'chicken', 'radio', 'agree', 'roomie', 'paris', 'ill', 'visual', 'uni', 'wee', 'flights', 'nom', 'hahah', 'yay', 'weee', 'ftw', 'straight', 'nah', 'wrote', 'deserved', 'bat', 'good time', 'wonderful', 'ben', 'enjoying', 'lady', 'cute', 'excited', 'just hours', 'beautiful', 'tee', 'fan', 'thanks', 'sweet', 'welcome', 'singing', 'congrats', 'ticke

In [14]:
print(clf.predict(vectorizer.transform(["It's a sad movie but very good"])))
print(clf.predict(vectorizer.transform(["Waste of my time"])))
print(clf.predict(vectorizer.transform(["It is not what like"])))
print(clf.predict(vectorizer.transform(["It is not what I m looking for"])))

[-1]
[-1]
[1]
[-1]


In [15]:
from sklearn.linear_model import SGDClassifier
svm_clf = SGDClassifier(loss="hinge", penalty='l2')
svm_clf.fit(train_data, train_labels)
svm_preds_test = svm_clf.predict(test_data)
svm_errs_test = np.sum((svm_preds_test > 0.0) != (test_labels > 0.0))
print("Test error: ", float(svm_errs_test)/len(test_labels))

Test error:  0.344


In [16]:
print(svm_clf.predict(vectorizer.transform(["It's a sad movie but very good"])))
print(svm_clf.predict(vectorizer.transform(["Waste of my time"])))
print(svm_clf.predict(vectorizer.transform(["This is not what I like"])))
print(svm_clf.predict(vectorizer.transform(["It is not what I am looking for"])))

[-1]
[-1]
[1]
[-1]


In [17]:
max_review_length = 200
tokenizer = Tokenizer(num_words=10000,  #max no. of unique words to keep
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', 
                      lower=True #convert to lower case
                     )
tokenizer.fit_on_texts(content2)

X = tokenizer.texts_to_sequences(content2)
X = pad_sequences(X, maxlen= max_review_length)
print('Shape of data tensor:', X.shape)

Y=pd.get_dummies(y).values

np.random.seed(0)
test_inds = np.append(np.random.choice((np.where(y==-1))[0], 250, replace=False), np.random.choice((np.where(y==1))[0], 250, replace=False))
train_inds = list(set(range(len(labels))) - set(test_inds))
train_data = X[train_inds,]
train_labels = Y[train_inds]
test_data = X[test_inds,]
test_labels = Y[test_inds]

Shape of data tensor: (5000, 200)


In [18]:
EMBEDDING_DIM = 200
model = Sequential()
model.add(Embedding(10000, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(250, dropout=0.2,return_sequences=True))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 200)          2000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 200)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 200, 250)          451000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               140400    
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 2,591,602
Trainable params: 2,591,602
Non-trainable params: 0
______________________________________________

In [19]:
epochs = 2
batch_size = 40
model.fit(train_data, train_labels, 
          epochs=epochs, 
          batch_size=batch_size,
          validation_split=0.1)

Epoch 1/2
102/102 [==============================] - 117s 1s/step - loss: 0.6387 - accuracy: 0.6257 - val_loss: 0.5661 - val_accuracy: 0.7000
Epoch 2/2
102/102 [==============================] - 113s 1s/step - loss: 0.3953 - accuracy: 0.8286 - val_loss: 0.5862 - val_accuracy: 0.6978


In [20]:
loss, acc = model.evaluate(test_data, test_labels, verbose=2,
                            batch_size=batch_size)
print(f"Loss: {loss}")
print(f"Validation accuracy: {acc}")

13/13 - 3s - loss: 0.5815 - accuracy: 0.6880 - 3s/epoch - 229ms/step
Loss: 0.581498384475708
Validation accuracy: 0.6880000233650208


In [21]:
outcome_labels = ['Negative', 'Positive']
new = ["test"]
 
def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(seq, maxlen=max_review_length)
    pred = model.predict(padded)
    print("Probability distribution: ", pred)
    print(f"Is this a Positive or Negative message? '{new[0]}'")
    print(outcome_labels[np.argmax(pred)])

predict_sentiment(new)

1/1 [==============================] - 1s 924ms/step
Probability distribution:  [[0.6206971  0.37930298]]
Is this a Positive or Negative message? 'test'
Negative


In [38]:
predict_sentiment(['dr bixler'])

1/1 [==============================] - 0s 45ms/step
Probability distribution:  [[0.4545355  0.54546446]]
Is this a Positive or Negative message? 'dr bixler'
Positive
